# Pandapower Standard Type Library

Pandapower provides a standard type library that allows the comfortable managing of type data for lines and transformers.

The following examples are all based on lines, but can be equally used for the transformer database by specifying element="trafo" instead of element="line" in all standard type functions.

## Basic Pandapower Standard Types

Pandapower comes with generic standard types that you can use to build generic networks:

In [1]:
import pandapower as pp
net = pp.create_empty_network()
print(pp.available_std_types(net, element="line"))

                                 c_nf_per_km imax_ka q_mm2 r_ohm_per_km type  \
149-AL1/24-ST1A 10.0                   11.25    0.47   149        0.194   ol   
149-AL1/24-ST1A 110.0                   8.75    0.47   149        0.194   ol   
149-AL1/24-ST1A 20.0                    10.5    0.47   149        0.194   ol   
15-AL1/3-ST1A 0.4                         11   0.105    16       1.8769   ol   
184-AL1/30-ST1A 110.0                    8.8   0.535   184       0.1571   ol   
184-AL1/30-ST1A 20.0                   10.75   0.535   184       0.1571   ol   
24-AL1/4-ST1A 0.4                      11.25    0.14    24       1.2012   ol   
243-AL1/39-ST1A 110.0                      9   0.645   243       0.1188   ol   
243-AL1/39-ST1A 20.0                      11   0.645   243       0.1188   ol   
305-AL1/39-ST1A 110.0                    9.2    0.74   305       0.0949   ol   
48-AL1/8-ST1A 0.4                       12.2    0.21    48       0.5939   ol   
48-AL1/8-ST1A 10.0                      

If you create a line with the create_line function, the element parameters are taken from this library:

In [2]:
b1 = pp.create_bus(net, vn_kv=.4)
b2 = pp.create_bus(net, vn_kv=.4)
lid = pp.create_line(net, from_bus=b1, to_bus=b2, length_km=0.1, std_type="NAYY 4x50 SE", name="test_line")
print(net.line.loc[lid])

name               test_line
std_type        NAYY 4x50 SE
from_bus                   0
to_bus                     1
length_km                0.1
r_ohm_per_km           0.642
x_ohm_per_km           0.083
c_nf_per_km              210
imax_ka                0.142
df                         1
parallel                   1
type                      cs
in_service              True
Name: 0, dtype: object


## Create Individual Standard Types

Additionally to the basic pandapower standard types, you can define individual standard types for you pandapower network.

To create a new standard type you need at least the following parameters:

**For Lines**:
- r_ohm_per_km
- x_ohm_per_km
- c_nf_per_km
- imax_ka

**For Transformers:**
- sn_kva
- vn_hv_kv
- vn_lv_kv
- vsc_percent
- vscr_percent
- i0_percent
- pfe_kw
- shift_degree

To define the parameters, create a dictionary with the style {"parameter1": parameter_value1, "parameter2": paramter_value2,...} and save the standard type to your network with the create_std_type function:

In [3]:
net = pp.create_empty_network()
test_type = {"r_ohm_per_km": 0.01, "x_ohm_per_km": 0.02, "c_nf_per_km": 10, "imax_ka": 0.4, "type": "cs"}
pp.create_std_type(net, name="test_type", data=test_type, element="line")
print(pp.available_std_types(net, element="line"))

                                 c_nf_per_km imax_ka q_mm2 r_ohm_per_km type  \
149-AL1/24-ST1A 10.0                   11.25    0.47   149        0.194   ol   
149-AL1/24-ST1A 110.0                   8.75    0.47   149        0.194   ol   
149-AL1/24-ST1A 20.0                    10.5    0.47   149        0.194   ol   
15-AL1/3-ST1A 0.4                         11   0.105    16       1.8769   ol   
184-AL1/30-ST1A 110.0                    8.8   0.535   184       0.1571   ol   
184-AL1/30-ST1A 20.0                   10.75   0.535   184       0.1571   ol   
24-AL1/4-ST1A 0.4                      11.25    0.14    24       1.2012   ol   
243-AL1/39-ST1A 110.0                      9   0.645   243       0.1188   ol   
243-AL1/39-ST1A 20.0                      11   0.645   243       0.1188   ol   
305-AL1/39-ST1A 110.0                    9.2    0.74   305       0.0949   ol   
48-AL1/8-ST1A 0.4                       12.2    0.21    48       0.5939   ol   
48-AL1/8-ST1A 10.0                      

As you can see the test_type is now available in the standard type library, and you can use it to create a line:

In [4]:
b1 = pp.create_bus(net, vn_kv=.4)
b2 = pp.create_bus(net, vn_kv=.4)
lid = pp.create_line(net, from_bus=b1, to_bus=b2, length_km=0.1, std_type="test_type", name="test_line")
print(net.line.loc[lid])

name            test_line
std_type        test_type
from_bus                0
to_bus                  1
length_km             0.1
r_ohm_per_km         0.01
x_ohm_per_km         0.02
c_nf_per_km            10
imax_ka               0.4
df                      1
parallel                1
type                   cs
in_service           True
Name: 0, dtype: object


## Local Standard Type Libraries

Standard tpyes are saved with the network if you use the pp.to_pickle function and are therefore still available if you load the network again.

If you have a set of standard types that you want to save locally and apply to multiple networks, you can create a set of standard type parameters as a dictionary in the style {"std_type1: {"r_ohm_per_km": r_ohm_per_km,...}, "std_type2": {"r_ohm_per_km": r_ohm_per_km,...},...} and add it to the network with the create_std_types function:

In [5]:
net = pp.create_empty_network()
linetypes = {"typ1": {"r_ohm_per_km": 0.01, "x_ohm_per_km": 0.02, "c_nf_per_km": 10, "imax_ka": 0.4, "type": "cs"},
             "typ2": {"r_ohm_per_km": 0.015, "x_ohm_per_km": 0.01, "c_nf_per_km": 30, "imax_ka": 0.3, "type": "cs"}}
pp.create_std_types(net, data=linetypes, element="line")
print(pp.available_std_types(net))

                                 c_nf_per_km imax_ka q_mm2 r_ohm_per_km type  \
149-AL1/24-ST1A 10.0                   11.25    0.47   149        0.194   ol   
149-AL1/24-ST1A 110.0                   8.75    0.47   149        0.194   ol   
149-AL1/24-ST1A 20.0                    10.5    0.47   149        0.194   ol   
15-AL1/3-ST1A 0.4                         11   0.105    16       1.8769   ol   
184-AL1/30-ST1A 110.0                    8.8   0.535   184       0.1571   ol   
184-AL1/30-ST1A 20.0                   10.75   0.535   184       0.1571   ol   
24-AL1/4-ST1A 0.4                      11.25    0.14    24       1.2012   ol   
243-AL1/39-ST1A 110.0                      9   0.645   243       0.1188   ol   
243-AL1/39-ST1A 20.0                      11   0.645   243       0.1188   ol   
305-AL1/39-ST1A 110.0                    9.2    0.74   305       0.0949   ol   
48-AL1/8-ST1A 0.4                       12.2    0.21    48       0.5939   ol   
48-AL1/8-ST1A 10.0                      

The linetypes dictionary can be easily saved in a local project folder (e.g. using jsonpickle) and reloaded once it is needed. 

Another possibility is to copy the standard type library of an existing network:

In [6]:
new_net = pp.create_empty_network()
pp.copy_std_types(from_net=net, to_net=new_net, element="line")
print(pp.available_std_types(new_net))

                                 c_nf_per_km imax_ka q_mm2 r_ohm_per_km type  \
149-AL1/24-ST1A 10.0                   11.25    0.47   149        0.194   ol   
149-AL1/24-ST1A 110.0                   8.75    0.47   149        0.194   ol   
149-AL1/24-ST1A 20.0                    10.5    0.47   149        0.194   ol   
15-AL1/3-ST1A 0.4                         11   0.105    16       1.8769   ol   
184-AL1/30-ST1A 110.0                    8.8   0.535   184       0.1571   ol   
184-AL1/30-ST1A 20.0                   10.75   0.535   184       0.1571   ol   
24-AL1/4-ST1A 0.4                      11.25    0.14    24       1.2012   ol   
243-AL1/39-ST1A 110.0                      9   0.645   243       0.1188   ol   
243-AL1/39-ST1A 20.0                      11   0.645   243       0.1188   ol   
305-AL1/39-ST1A 110.0                    9.2    0.74   305       0.0949   ol   
48-AL1/8-ST1A 0.4                       12.2    0.21    48       0.5939   ol   
48-AL1/8-ST1A 10.0                      

## Add Additional Parameters

You can define additional parameters in the standard type library and load them into pandapower. For example, if you want to run a reliability analysis for which you need a hazard rate for each line, you will want to add the hazard rate "h_per_km" for your individual standard type:

In [7]:
import pandapower as pp
net = pp.create_empty_network()
test_type = {"r_ohm_per_km": 0.01, "x_ohm_per_km": 0.02, "c_nf_per_km": 10, "imax_ka": 0.4, "type": "cs", "h_per_km": 0.01}
pp.create_std_type(net, name="test_type", data=test_type, element="line")
print(pp.available_std_types(net, element="line"))

                                 c_nf_per_km h_per_km imax_ka q_mm2  \
149-AL1/24-ST1A 10.0                   11.25      NaN    0.47   149   
149-AL1/24-ST1A 110.0                   8.75      NaN    0.47   149   
149-AL1/24-ST1A 20.0                    10.5      NaN    0.47   149   
15-AL1/3-ST1A 0.4                         11      NaN   0.105    16   
184-AL1/30-ST1A 110.0                    8.8      NaN   0.535   184   
184-AL1/30-ST1A 20.0                   10.75      NaN   0.535   184   
24-AL1/4-ST1A 0.4                      11.25      NaN    0.14    24   
243-AL1/39-ST1A 110.0                      9      NaN   0.645   243   
243-AL1/39-ST1A 20.0                      11      NaN   0.645   243   
305-AL1/39-ST1A 110.0                    9.2      NaN    0.74   305   
48-AL1/8-ST1A 0.4                       12.2      NaN    0.21    48   
48-AL1/8-ST1A 10.0                      10.1      NaN    0.21    48   
48-AL1/8-ST1A 20.0                       9.5      NaN    0.21    48   
490-AL

You can see that the parameter "h_per_km" is stored in the standard type database for the new standard type.

We now create two lines, one of the new "test_type" and one with a standard type that does not have the parameter "h_per_km".

In [8]:
b1 = pp.create_bus(net, vn_kv=.4)
b2 = pp.create_bus(net, vn_kv=.4)
l1 = pp.create_line(net, from_bus=b1, to_bus=b2, length_km=0.1, std_type="test_type", name="test_line1")
l2 = pp.create_line(net, from_bus=b1, to_bus=b2, length_km=0.1, std_type="NAYY 4x50 SE", name="test_line2")
print(net.line)

         name      std_type  from_bus  to_bus  length_km  r_ohm_per_km  \
0  test_line1     test_type         0       1        0.1         0.010   
1  test_line2  NAYY 4x50 SE         0       1        0.1         0.642   

   x_ohm_per_km  c_nf_per_km  imax_ka   df  parallel type in_service  
0         0.020         10.0    0.400  1.0         1   cs       True  
1         0.083        210.0    0.142  1.0         1   cs       True  


The parameter "h_per_km" ist not in the line table, since it is not a pandapower standard parameter. You can however load the parameter from the database with the parameter_from_std_type function:

In [9]:
pp.parameter_from_std_type(net, parameter="h_per_km")
print(net.line)

         name      std_type  from_bus  to_bus  length_km  r_ohm_per_km  \
0  test_line1     test_type         0       1        0.1         0.010   
1  test_line2  NAYY 4x50 SE         0       1        0.1         0.642   

   x_ohm_per_km  c_nf_per_km  imax_ka   df  parallel type in_service h_per_km  
0         0.020         10.0    0.400  1.0         1   cs       True     0.01  
1         0.083        210.0    0.142  1.0         1   cs       True     None  


Pandapower adds the parameter to the linetable where it is available. The second line does not have a value for h_per_km, since it is not specified in the standard type database. You can however define a fallback (fill) value for lines that do not have a standard type or that have a standard type for which the desired parameter is not available:

In [10]:
pp.parameter_from_std_type(net, parameter="h_per_km", fill=0.02)
print(net.line)

         name      std_type  from_bus  to_bus  length_km  r_ohm_per_km  \
0  test_line1     test_type         0       1        0.1         0.010   
1  test_line2  NAYY 4x50 SE         0       1        0.1         0.642   

   x_ohm_per_km  c_nf_per_km  imax_ka   df  parallel type in_service h_per_km  
0         0.020         10.0    0.400  1.0         1   cs       True     0.01  
1         0.083        210.0    0.142  1.0         1   cs       True     0.02  


## Change Standard Type

The change_std_type function allows changing the standard type of an element.

In [11]:
net = pp.create_empty_network()
b1 = pp.create_bus(net, vn_kv=.4)
b2 = pp.create_bus(net, vn_kv=.4)
l2 = pp.create_line(net, from_bus=b1, to_bus=b2, length_km=0.1, std_type="NAYY 4x50 SE", name="test_line")
print(net.line)

        name      std_type  from_bus  to_bus  length_km  r_ohm_per_km  \
0  test_line  NAYY 4x50 SE         0       1        0.1         0.642   

   x_ohm_per_km  c_nf_per_km  imax_ka   df  parallel type in_service  
0         0.083        210.0    0.142  1.0         1   cs       True  


In [12]:
pp.change_std_type(net, l2, "NAYY 4x150 SE", element="line")
print(net.line)

        name       std_type  from_bus  to_bus  length_km  r_ohm_per_km  \
0  test_line  NAYY 4x150 SE         0       1        0.1         0.208   

   x_ohm_per_km  c_nf_per_km  imax_ka   df  parallel type in_service  
0          0.08        261.0     0.27  1.0         1   cs       True  


## Find Standard Type

It is possible to search for a standard type given a set of parameters:

In [13]:
print(pp.find_std_type_by_parameter(net, data={"r_ohm_per_km": 0.1188, "x_ohm_per_km": 0.39}))

['243-AL1/39-ST1A 110.0']


You can also define a tolerance epsilon to find types with similar parameters:

In [14]:
print(pp.find_std_type_by_parameter(net, data={"r_ohm_per_km": 0.12, "x_ohm_per_km": 0.40}, epsilon=.02))

['243-AL1/39-ST1A 110.0']
